# Análise de Correlação e Associação em Dados de Vendas

Este notebook demonstra, de forma prática, como escolher e aplicar **métricas estatísticas adequadas** para analisar relações entre variáveis **quantitativas** e **categóricas**.


## 1. Contexto e objetivo
Em análises de vendas, é comum querer entender se variáveis como **quantidade vendida**, **preço unitário** e **valor total** se relacionam entre si.

**Objetivos**:
- Avaliar correlação entre variáveis **quantitativas** (Pearson e Spearman)
- Avaliar associação entre variáveis **categóricas** (Qui-quadrado e Cramér’s V)
- Comparar uma variável **categórica vs. quantitativa** (boxplot e ANOVA)
- Consolidar conclusões claras (boa prática para portfólio)


## 2. Dicionário de dados (variáveis)
- `quantidade` (quantitativa discreta): unidades vendidas
- `preco_unitario` (quantitativa contínua): preço por unidade
- `valor_total` (quantitativa contínua): `quantidade * preco_unitario`
- `tipo_produto` (categórica nominal): categoria do produto

> Observação: o dataset abaixo é **sintético** (gerado por código) para fins didáticos. O mesmo fluxo se aplica a dados reais.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuração para reprodutibilidade
np.random.seed(42)


## 3. Geração do dataset (sintético)
Geramos `n = 300` registros para simular vendas com três tipos de produto. A semente (`seed`) garante que os resultados sejam reprodutíveis.

In [ ]:
n = 300

# Quantidade: valores inteiros (contagem)
quantidade = np.random.randint(1, 50, n)

# Preço unitário: valores contínuos (medição)
preco_unitario = np.random.uniform(10, 50, n)

# Valor total: definição determinística para exemplificar relação com quantidade e preço
valor_total = quantidade * preco_unitario

# Variável categórica nominal
tipo_produto = np.random.choice(
    ['Grama Bermuda', 'Grama Esmeralda', 'Hidrossemeadura'],
    size=n
)

df = pd.DataFrame({
    'quantidade': quantidade,
    'preco_unitario': preco_unitario,
    'valor_total': valor_total,
    'tipo_produto': tipo_produto
})

df.head()


## 4. Entendimento e validação dos dados
Nesta etapa, validamos tipos, distribuição e eventuais valores ausentes. Em dados reais, essa etapa também inclui tratamento de outliers e consistência de regras de negócio.

In [ ]:
df.info()


In [ ]:
df.describe()


## 5. Correlação entre variáveis quantitativas
Aqui usamos:
- **Pearson**: mede relação **linear**
- **Spearman**: mede relação **monotônica** (baseada em ranking; mais robusta a outliers)

> Importante: correlação mede associação estatística, **não causalidade**.


### 5.1 Correlação de Pearson (linear)
Interpretação do coeficiente: varia de **-1 a +1**. Valores próximos de +1 indicam relação positiva forte; próximos de -1 indicam relação negativa forte; próximos de 0 indicam ausência de relação linear evidente.

In [ ]:
corr_pearson = df[['quantidade', 'preco_unitario', 'valor_total']].corr(method='pearson')
corr_pearson


### 5.2 Correlação de Spearman (monotônica)
Útil quando a relação não é estritamente linear ou quando há suspeita de outliers influenciando a correlação.

In [ ]:
corr_spearman = df[['quantidade', 'preco_unitario', 'valor_total']].corr(method='spearman')
corr_spearman


### 5.3 Visualização (sanity check)
Mesmo com correlação calculada, é boa prática **visualizar** a relação para evitar conclusões equivocadas.

In [ ]:
plt.figure()
plt.scatter(df['quantidade'], df['valor_total'])
plt.xlabel('Quantidade vendida')
plt.ylabel('Valor total')
plt.title('Quantidade vs. Valor total')
plt.show()


## 6. Associação entre variáveis categóricas
Para variáveis **categóricas nominais**, não aplicamos Pearson/Spearman. Em vez disso, usamos:
- **Qui-quadrado**: testa se existe associação (significância)
- **Cramér’s V**: mede a **força** da associação (0 a 1)


### 6.1 Criação de uma segunda variável categórica
Para exemplificar a análise categórica, adicionamos `forma_pagamento` (Pix/Crédito/Boleto).

In [ ]:
df['forma_pagamento'] = np.random.choice(
    ['Pix', 'Crédito', 'Boleto'],
    size=len(df)
)

df[['tipo_produto', 'forma_pagamento']].head()


### 6.2 Tabela de contingência
A base do Qui-quadrado é a tabela que cruza as categorias.

In [ ]:
tabela = pd.crosstab(df['tipo_produto'], df['forma_pagamento'])
tabela


### 6.3 Teste Qui-quadrado
Hipóteses:
- H0: as variáveis são independentes (sem associação)
- H1: existe associação

Regra prática: se `p < 0.05`, rejeitamos H0 e consideramos evidência de associação.

In [ ]:
from scipy.stats import chi2_contingency

chi2, p_val, dof, expected = chi2_contingency(tabela)
chi2, p_val


### 6.4 Cramér’s V (força da associação)
O Qui-quadrado indica se há evidência de associação. O **Cramér’s V** quantifica a força (0 a 1).

In [ ]:
import numpy as np

n_obs = tabela.to_numpy().sum()
cramers_v = np.sqrt(chi2 / (n_obs * (min(tabela.shape) - 1)))
cramers_v


Interpretação prática (regra geral, não absoluta):
- 0.00–0.10: muito fraca
- 0.10–0.30: fraca
- 0.30–0.50: moderada
- > 0.50: forte


## 7. Categórica vs. quantitativa
Quando queremos saber se uma variável quantitativa muda conforme grupos categóricos, usamos comparação de distribuições e testes como **ANOVA** (para 3+ grupos).

In [ ]:
df.groupby('tipo_produto')['valor_total'].describe()


In [ ]:
df.boxplot(column='valor_total', by='tipo_produto')
plt.title('Valor total por tipo de produto')
plt.suptitle('')
plt.xlabel('Tipo de produto')
plt.ylabel('Valor total')
plt.show()


### 7.1 ANOVA (diferença de médias entre grupos)
Hipóteses:
- H0: médias iguais entre todos os grupos
- H1: pelo menos um grupo difere

Regra prática: se `p < 0.05`, há evidência de diferença entre grupos.

In [ ]:
from scipy.stats import f_oneway

grupos = [df.loc[df['tipo_produto'] == tp, 'valor_total'] for tp in df['tipo_produto'].unique()]
f_stat, p_anova = f_oneway(*grupos)
f_stat, p_anova


## 8. Conclusões
- **Quantidade vs. valor_total** tende a apresentar correlação positiva relevante, dado que `valor_total = quantidade * preco_unitario`.
- **Preço_unitário vs. quantidade** tende a ser fraco/nulo neste dataset sintético, pois foram gerados de forma independente.
- Para **categóricas**, utilizamos Qui-quadrado (significância) e Cramér’s V (força), evitando o uso inadequado de Pearson/Spearman.
- Para **categórica vs. quantitativa**, o boxplot e a ANOVA ajudam a avaliar diferenças entre grupos.

**Próximos passos sugeridos**:
- Repetir a análise com um dataset real (CSV)
- Adicionar tratamento de outliers e checagens de normalidade
- Evoluir para regressão linear (explicar `valor_total` por `quantidade` e `preco_unitario`)
